In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mnist-original/mnist-original.mat


In [2]:
!pip install scipy -q

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import scipy.io as sc

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
mat = sc.loadmat('/kaggle/input/mnist-original/mnist-original.mat')
mnist_data = torch.tensor(mat["data"].T,dtype = torch.float32)
mnist_label = torch.tensor(mat["label"][0],dtype = torch.long)

In [6]:
class dataset(Dataset):
    def __init__(self, data,labels):
        self.data = data
        self. labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        return sample, label

In [7]:
mnist_dataset = dataset(mnist_data, mnist_label)

# Create DataLoader
train_loader = DataLoader(mnist_dataset, batch_size=128, shuffle=True)


In [8]:
class VAE(nn.Module):
    def __init__(self, input_dim=784, hidden_dim=500, latent_dim=30):
        super(VAE, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, latent_dim * 2)  # Output both μ and log(σ^2)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim),
            nn.Sigmoid()  # Output values in [0,1]
        )

    def reparameterize(self, mu, log_var):
        """Reparameterization trick: z = mu + sigma * epsilon"""
        log_var = torch.clamp(log_var, min=-10, max=10)  # Avoid extreme values
        std = torch.exp(0.5 * log_var)
        epsilon = torch.randn_like(std)
        return mu + std * epsilon

    def forward(self, x):
        # Flatten input
        x = x.view(-1, 784)

        # Encoder: Get μ and log(σ^2)
        h = self.encoder(x)
        mu, log_var = torch.chunk(h, 2, dim=1)  # Split into two parts

        # Sample latent vector
        z = self.reparameterize(mu, log_var)

        # Decoder: Reconstruct x
        x_recon = self.decoder(z)
        return x_recon, mu, log_var


In [9]:
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [10]:
def loss_function(x_recon, x, mu, log_var):
    recon_loss = nn.functional.mse_loss(x_recon, x.view(-1, 784), reduction="sum")
    kl_div = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())  # KL divergence
    return recon_loss + kl_div

In [11]:
num_epochs = 20
criterion = nn.BCELoss(reduction='sum')  # BCE loss

for epoch in range(num_epochs):
    epoch_recon_loss = 0.0
    epoch_kl_loss = 0.0
    num_batches = len(train_loader)

    for images, _ in train_loader:
        # Normalize images to [0, 1] and flatten
        images = images.view(-1, 784).to(device) / 255.0
        
        # Forward pass
        outputs, mu, log_var = model(images)
        
        # Compute losses
        reconstruction_loss = criterion(outputs, images)
        kl_divergence = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())

        # Accumulate losses
        epoch_recon_loss += reconstruction_loss.item()
        epoch_kl_loss += kl_divergence.item()

        # Backpropagation
        optimizer.zero_grad()
        loss = reconstruction_loss + kl_divergence
        loss.backward()
        optimizer.step()
    
    # Print average losses for the epoch
    avg_recon_loss = epoch_recon_loss / num_batches
    avg_kl_loss = epoch_kl_loss / num_batches
    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Recon Loss: {avg_recon_loss:.4f}, "
          f"KL Loss: {avg_kl_loss:.4f}, "
          f"Total Loss: {avg_recon_loss + avg_kl_loss:.4f}")

Epoch [1/20], Recon Loss: 17972.8176, KL Loss: 2195.5059, Total Loss: 20168.3235
Epoch [2/20], Recon Loss: 12146.8802, KL Loss: 3110.1204, Total Loss: 15257.0006
Epoch [3/20], Recon Loss: 11137.1394, KL Loss: 3286.0195, Total Loss: 14423.1589
Epoch [4/20], Recon Loss: 10715.4163, KL Loss: 3343.7575, Total Loss: 14059.1738
Epoch [5/20], Recon Loss: 10482.8356, KL Loss: 3371.8985, Total Loss: 13854.7341
Epoch [6/20], Recon Loss: 10334.1455, KL Loss: 3387.4702, Total Loss: 13721.6156
Epoch [7/20], Recon Loss: 10232.6645, KL Loss: 3394.9929, Total Loss: 13627.6574
Epoch [8/20], Recon Loss: 10149.4929, KL Loss: 3400.6267, Total Loss: 13550.1196
Epoch [9/20], Recon Loss: 10089.4172, KL Loss: 3406.0163, Total Loss: 13495.4336
Epoch [10/20], Recon Loss: 10037.3096, KL Loss: 3408.1628, Total Loss: 13445.4724
Epoch [11/20], Recon Loss: 10001.7252, KL Loss: 3413.3609, Total Loss: 13415.0861
Epoch [12/20], Recon Loss: 9961.1294, KL Loss: 3412.1334, Total Loss: 13373.2628
Epoch [13/20], Recon Loss: